<a href="https://colab.research.google.com/github/yoonyoung27/prac_class/blob/main/personal_project_M1F4_%EC%A7%80%EC%9C%A4%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#신용카드 사용자 연체 예측 AI 경진대회

https://www.dacon.io/competitions/official/235713/overview/description/

###✓ Import data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd

In [52]:
user_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
user_train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [53]:
user_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
user_test.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


###✓ Information of data

In [54]:
user_train.isnull().sum()

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

1. user_train의 occyp_type 항목에는 8171개의 결측치가 존재한다

In [55]:
user_test.isnull().sum()

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       3152
family_size         0
begin_month         0
dtype: int64

2. user_train의 occyp_type 항목에는 3152개의 결측치가 존재한다

In [56]:
unique_len = user_train.apply(lambda x: len(x.unique()))
unique_len

index            26457
gender               2
car                  2
reality              2
child_num            9
income_total       249
income_type          5
edu_type             5
family_type          5
house_type           6
DAYS_BIRTH        6621
DAYS_EMPLOYED     3470
FLAG_MOBIL           1
work_phone           2
phone                2
email                2
occyp_type          19
family_size         10
begin_month         61
credit               3
dtype: int64

3. user_train의 각 항목의 값은 위의 개수와 같이 분류되어 있으며, 그 값은 아래와 같다

In [57]:
user_train.apply(lambda x: x.unique())

index            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
gender                                                      [F, M]
car                                                         [N, Y]
reality                                                     [N, Y]
child_num                            [0, 1, 2, 3, 4, 5, 14, 19, 7]
income_total     [202500.0, 247500.0, 450000.0, 157500.0, 27000...
income_type      [Commercial associate, Working, State servant,...
edu_type         [Higher education, Secondary / secondary speci...
family_type      [Married, Civil marriage, Separated, Single / ...
house_type       [Municipal apartment, House / apartment, With ...
DAYS_BIRTH       [-13899, -11380, -19087, -15088, -15037, -1341...
DAYS_EMPLOYED    [-4709, -1540, -4434, -2092, -2105, -4996, -19...
FLAG_MOBIL                                                     [1]
work_phone                                                  [0, 1]
phone                                                       [0

###✓ Prepare data

1. user_train에서 결측치가 존재하는 occyp_type 항목을 제거한다

In [58]:
user_train = user_train.drop("occyp_type", axis=1)

2. user_train에서 결측치가 존재하는 occyp_type 항목을 제거한다

In [59]:
user_test = user_test.drop("occyp_type", axis=1)

3. ①. 값의 분류가 2개 이하, 2개 초과 10개 이하, 10개 초과인 항목을 묶는다

In [60]:
group_1 = unique_len[unique_len <= 2].index
group_1

Index(['gender', 'car', 'reality', 'FLAG_MOBIL', 'work_phone', 'phone',
       'email'],
      dtype='object')

In [61]:
group_2 = unique_len[(unique_len > 2)&(unique_len <= 10)].index
group_2

Index(['child_num', 'income_type', 'edu_type', 'family_type', 'house_type',
       'family_size', 'credit'],
      dtype='object')

In [62]:
group_3 = unique_len[(unique_len > 10)].index
group_3

Index(['index', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'occyp_type',
       'begin_month'],
      dtype='object')

3. ②. group_1의 항목을 이진변수로 인코딩한다

In [63]:
user_train[group_1]

,gender,car,reality,FLAG_MOBIL,work_phone,phone,email
0,F,N,N,1,0,0,0
1,F,N,Y,1,0,0,1
2,M,Y,Y,1,0,1,0
3,F,N,Y,1,0,1,0
4,F,Y,Y,1,0,0,0
...,...,...,...,...,...,...,...
26452,F,N,N,1,0,0,0
26453,F,N,Y,1,0,0,0
26454,F,Y,N,1,0,0,0
26455,M,N,Y,1,0,0,0


* gender, car, reality 항목을 이진변수로 인코딩한다.

In [64]:
user_train['gender'] = user_train['gender'].replace(['F','M'], [0,  1])
user_train['car'] = user_train['car'].replace(['N', 'Y'], [0, 1])
user_train['reality'] = user_train['reality'].replace(['N', 'Y'], [0, 1])

In [65]:
user_train[group_1]

,gender,car,reality,FLAG_MOBIL,work_phone,phone,email
0,0,0,0,1,0,0,0
1,0,0,1,1,0,0,1
2,1,1,1,1,0,1,0
3,0,0,1,1,0,1,0
4,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...
26452,0,0,0,1,0,0,0
26453,0,0,1,1,0,0,0
26454,0,1,0,1,0,0,0
26455,1,0,1,1,0,0,0
